In [20]:
import browser_cookie3
import requests
from pandas import ExcelWriter
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup

cookies = browser_cookie3.chrome(domain_name='.walmart.com')
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
}

with requests.session() as s:
    s.cookies = cookies
    s.headers = headers
    url = 'https://www.walmart.com/cart'
    resp = s.get(url)
    print('resp text contains shopping cart data')

resp text contains shopping cart data


In [21]:
soup=BeautifulSoup(resp.content, 'html.parser')
with open('walmartcart.html','w') as f:
    f.write(str(x))

In [61]:
items=soup.find_all('div',{'class':'cart-item-name js-product-title'})
itemlist=['https://www.walmart.com/ip/' + i.a['data-us-item-id'] for i in items]

In [62]:
def get_allitems_detail():
    imgurl=soup.find_all('span',{'class':'asset-container'})
    imgslinks=[i.img['src'] for i in imgurl]
    
    title=soup.find_all('div',{'class':'cart-item-name js-product-title'})
    titles=[i.text.strip()[:16] for i in title]
    
    price=soup.find_all('div',{'data-automation-id':'cart-item-primary-price'})
    prices=[i.find('span',{'class':'visuallyhidden'}).text for i in price]

    item_dict ={ 
        'ProductLinks': itemlist,     
        'Title': titles,
                 'Price': prices,
            'ImageLinks': imgslinks,
           }
        
    return pd.DataFrame(item_dict)

In [63]:
get_allitems_detail()

,ProductLinks,Title,Price,ImageLinks
0,https://www.walmart.com/ip/513876975,Better Homes & G,$9.97,https://i5.walmartimages.com/asr/3caf28ad-94c1...
1,https://www.walmart.com/ip/984896848,Mainstays Decora,$9.97,https://i5.walmartimages.com/asr/1468b57c-50ad...
2,https://www.walmart.com/ip/312959360,Mainstays Buffal,$11.87,https://i5.walmartimages.com/asr/a87f79b7-c428...
3,https://www.walmart.com/ip/935590307,Mainstays Giant,$11.80,https://i5.walmartimages.com/asr/5e2406a9-5783...
4,https://www.walmart.com/ip/650450071,Mainstays Gingha,$10.97,https://i5.walmartimages.com/asr/9d959328-f81f...
5,https://www.walmart.com/ip/567938229,Better Homes & G,$29.99,https://i5.walmartimages.com/asr/ca6da3b2-3944...
6,https://www.walmart.com/ip/651533651,Subrtex Stretch,$21.99,https://i5.walmartimages.com/asr/a5cae58e-6439...
7,https://www.walmart.com/ip/203125631,Mainstays Faux S,$7.88,https://i5.walmartimages.com/asr/325b47c3-5c7a...
8,https://www.walmart.com/ip/826783037,Phantoscope Soft,$24.99,https://i5.walmartimages.com/asr/343026b7-b74d...
9,https://www.walmart.com/ip/210595452,Mainstays Cordur,$10.88,https://i5.walmartimages.com/asr/185edd0a-8939...


In [64]:
writer = ExcelWriter('WalmartCartItems.xlsx')
get_allitems_detail().to_excel(writer,'Sheet5')
writer.save()